### Installation

In [ ]:
!pip install psycopg2
!pip install python-dotenv

!sudo apt-get update
!sudo apt-get install openvpn -y

### Load credentials and Variables

In [1]:
import psycopg2
from psycopg2 import sql

# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv()

# Get database credentials from environment variables
dbname = os.getenv("dbname")
engine = os.getenv("engine")
host = os.getenv("host")
password = os.getenv("password")
port = os.getenv("port")
username = os.getenv("username")
vpn_path = os.getenv("vpn_path")
api_key = os.getenv("api_key")

# Database connection parameters
db_params = {
    'dbname': dbname,
    'user': username,
    'password': password,
    'host': host,
    'port': port  # Add port if necessary
}





import subprocess

# Step 3: Start OpenVPN in the background and check the status
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

### Connection Troubleshooting

In [ ]:
import subprocess
import time

# Replace '/path/to/your/file.ovpn' with the actual path to your .ovpn file
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

# Collect logs for debugging
vpn_connected = False
for _ in range(30):  # Try for up to 30 seconds
    output = openvpn_process.stdout.readline().decode('utf-8')
    error_output = openvpn_process.stderr.readline().decode('utf-8')
    if "Initialization Sequence Completed" in output:
        vpn_connected = True
        break
    if error_output:
        print(f"Error: {error_output.strip()}")
    time.sleep(1)

if vpn_connected:
    print("VPN connection established successfully.")
else:
    print("Failed to establish VPN connection.")
    openvpn_process.terminate()
    raise RuntimeError("VPN connection failed")


In [ ]:
import subprocess
import time
import psycopg2

# Step 1: Install OpenVPN and dependencies
#!sudo apt-get update
#!sudo apt-get install -y openvpn easy-rsa

# Step 2: Remove existing TUN/TAP interfaces
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun0'])
except subprocess.CalledProcessError:
    pass
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun1'])
except subprocess.CalledProcessError:
    pass
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun2'])
except subprocess.CalledProcessError:
    pass
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun3'])
except subprocess.CalledProcessError:
    pass

# Step 3: Start OpenVPN in the background and check the status
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

# Collect logs for debugging
vpn_connected = False
for _ in range(30):  # Try for up to 30 seconds
    output = openvpn_process.stdout.readline().decode('utf-8')
    error_output = openvpn_process.stderr.readline().decode('utf-8')
    if "Initialization Sequence Completed" in output:
        vpn_connected = True
        break
    if error_output:
        print(f"Error: {error_output.strip()}")
    time.sleep(1)

if vpn_connected:
    print("VPN connection established successfully.")
else:
    print("Failed to establish VPN connection.")
    openvpn_process.terminate()
    raise RuntimeError("VPN connection failed")




### Establish Connection and Sample

In [ ]:
# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=username,
        password=password,
        host=host,
        port=port
    )
    cursor = connection.cursor()
    print("Connection to the database established successfully")

    # Execute a sample query
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"Database version: {db_version}")

    # Close the cursor and connection
    cursor.close()
    connection.close()
    print("Database connection closed")
except Exception as error:
    print(f"Error connecting to the database: {error}")


In [ ]:
import socket

def check_connection(host, port):
    try:
        socket.create_connection((host, port), timeout=10)
        print(f"Successfully connected to {host} on port {port}")
    except (socket.timeout, ConnectionRefusedError) as e:
        print(f"Failed to connect to {host} on port {port}: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

host = "app-postgres-hml.crzx8j5qnycp.us-east-1.rds.amazonaws.com"
port = 5432
check_connection(host, port)


### Obtain Data Schema Function

In [ ]:
import psycopg2
from psycopg2 import sql

def get_column_names_with_sample(schema_name, table_name, **db_params):
    try:
        # Database connection parameters
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # SQL query to fetch column names and first row values
        query = sql.SQL("""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_schema = %s AND table_name = %s
        """)

        cursor.execute(query, (schema_name, table_name))
        columns = [row[0] for row in cursor.fetchall()]

        # SQL query to fetch the first row of values
        query = sql.SQL("SELECT * FROM {}.{} LIMIT 1").format(
            sql.Identifier(schema_name),
            sql.Identifier(table_name)
        )

        cursor.execute(query)
        sample_row = cursor.fetchone()

        # Close the connection
        cursor.close()
        conn.close()

        # Combine columns with sample values
        if sample_row:
            column_samples = [f"Column: {columns[i]}, Sample Value: {sample_row[i]}" for i in range(len(columns))]
        else:
            column_samples = [f"Column: {column}, Sample Value: None" for column in columns]
        
        return "\n".join(column_samples)

    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
schema_name = 'app'
table_name = 'house'

schema = get_column_names_with_sample(schema_name, table_name, **db_params)
print(schema)


### Database Project Code Main

In [ ]:
try:
    # Establish a connection to the database
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    print("Connection to the database established successfully")

    # Execute a sample query to get the database version
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"Database version: {db_version}")

    # Query to list all schemas
    cursor.execute("""
    SELECT schema_name
    FROM information_schema.schemata
    ORDER BY schema_name;
    """)
    
    # Fetch all schemas
    schemas = cursor.fetchall()
    
    # Print schemas and their tables
    print("Schemas and their tables in the database:")
    for schema in schemas:
        schema_name = schema[0]
        print(f"\nSchema: {schema_name}")
        
        cursor.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = %s
        ORDER BY table_name;
        """, (schema_name,))
        
        tables = cursor.fetchall()
        
        if tables:
            for table in tables:
                print(f"  Table: {table[0]}")
        else:
            print("  No tables found in this schema.")

except Exception as error:
    print(f"Error connecting to the database or fetching tables: {error}")

finally:
    # Ensure cursor and connection are closed
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    print("Database connection closed")


In [ ]:
import psycopg2
import pandas as pd

# Connect to your postgres DB
conn = psycopg2.connect(**db_params)

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query to fetch a 5-row sample from the table 'house' in schema 'app'
cur.execute("SELECT * FROM app.house LIMIT 5")

# Retrieve the column names
colnames = [desc[0] for desc in cur.description]

# Retrieve the result of the query
rows = cur.fetchall()

# Convert to a pandas DataFrame
df = pd.DataFrame(rows, columns=colnames)

# Display the DataFrame
print(df)

# Close the cursor and connection
cur.close()
conn.close()


In [ ]:
import psycopg2
import pandas as pd

# Connect to your postgres DB
conn = psycopg2.connect(**db_params)

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query to fetch a 5-row sample from the table 'house' in schema 'app'
cur.execute("SELECT * FROM app.house LIMIT 5")

# Retrieve the column names
colnames = [desc[0] for desc in cur.description]

# Retrieve the result of the query
rows = cur.fetchall()

# Convert to a pandas DataFrame
df = pd.DataFrame(rows, columns=colnames)

# Set pandas display options to avoid truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.width', None)  # Adjust display width to accommodate the full content

# Display column names
print("Column Names:", colnames)

# Display the DataFrame
print(df.head())

# Close the cursor and connection
cur.close()
conn.close()


In [ ]:
from openai import OpenAI
client = OpenAI(api_key)

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are the link between the user and the database. You will answer the user's questions. This is for a property website mostly based in Orlando, Florida."
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_schema",
        "description": "Fetch schema of database table if needed",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ]
            }
          },
          "required": [
            "location"
          ]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "generate_sql_query",
        "description": "Parse user's natural language query and generate the appropiate SQL statement.",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ]
            }
          },
          "required": [
            "location"
          ]
        }
      }
    }
  ]
)